## Embeddings

En nuestro ejemplo anterior, trabajamos con vectores de bolsa de palabras de alta dimensionalidad con longitud `vocab_size`, y convertimos explícitamente los vectores de representación posicional de baja dimensionalidad en una representación dispersa *one-hot*. Esta representación *one-hot* no es eficiente en términos de memoria. Además, cada palabra se trata de manera independiente de las demás, por lo que los vectores codificados *one-hot* no expresan similitudes semánticas entre palabras.

En esta unidad, continuaremos explorando el conjunto de datos **News AG**. Para comenzar, carguemos los datos y obtengamos algunas definiciones de la unidad anterior.


In [ ]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/120000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/ag_news_subset/incomplete.4NAV83_1.0.0/ag_news_subset-train.tfrecord*...: …

Generating test examples...:   0%|          | 0/7600 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/ag_news_subset/incomplete.4NAV83_1.0.0/ag_news_subset-test.tfrecord*...:  …

Dataset ag_news_subset downloaded and prepared to /root/tensorflow_datasets/ag_news_subset/1.0.0. Subsequent calls will reuse this data.


In [ ]:
print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


### ¿Qué es un embedding?

La idea del **embedding** es representar palabras utilizando vectores densos de baja dimensionalidad que reflejan el significado semántico de la palabra. Más adelante discutiremos cómo construir embeddings de palabras significativos, pero por ahora, pensemos en los embeddings como una forma de reducir la dimensionalidad de un vector de palabras.

Así que, una capa de embedding toma una palabra como entrada y produce un vector de salida de un `embedding_size` especificado. En cierto sentido, es muy similar a una capa `Dense`, pero en lugar de tomar un vector codificado *one-hot* como entrada, es capaz de tomar un número que representa una palabra.

Al usar una capa de embedding como la primera capa en nuestra red, podemos cambiar de la bolsa de palabras a un modelo de **embedding bag**, donde primero convertimos cada palabra de nuestro texto en su correspondiente embedding, y luego calculamos alguna función agregada sobre todos esos embeddings, como `suma`, `promedio` o `máximo`.

![Imagen que muestra un clasificador de embedding para una secuencia de cinco palabras.](https://drive.google.com/uc?export=view&id=16eURCTqgEOuhwrI59YIAQGzX3ntkUWfX)

Nuestra red neuronal clasificadora consiste en las siguientes capas:

* Capa `TextVectorization`, que toma una cadena de texto como entrada y produce un tensor de números de tokens. Especificaremos un tamaño de vocabulario razonable `vocab_size`, e ignoraremos las palabras que se usan con menos frecuencia. La forma de entrada será 1, y la forma de salida será $n$, ya que obtendremos $n$ tokens como resultado, cada uno de ellos contendrá números del 0 a `vocab_size`.
* Capa `Embedding`, que toma $n$ números y reduce cada número a un vector denso de una longitud dada (100 en nuestro ejemplo). Así, el tensor de entrada de forma $n$ se transformará en un tensor de $n\times 100$.
* Capa de agregación, que toma el promedio de este tensor a lo largo del primer eje, es decir, calculará el promedio de todos los $n$ tensores de entrada que corresponden a diferentes palabras. Para implementar esta capa, utilizaremos una capa `Lambda` y le pasaremos la función para calcular el promedio. La salida tendrá una forma de 100, y será la representación numérica de toda la secuencia de entrada.
* Finalmente, un clasificador lineal `Dense`.


In [ ]:
# Definir el vectorizador
vectorizer = keras.layers.TextVectorization(max_tokens=vocab_size)

# Definir el modelo
model = keras.models.Sequential([
    # Capa de entrada que espera una secuencia de texto
    keras.Input(shape=(1,), dtype=tf.string),

    # Aplicar el vectorizador
    vectorizer,

    # Capa de embeddings para convertir tokens en vectores de baja dimensionalidad
    keras.layers.Embedding(vocab_size, 100),

    # Calcular el promedio de los embeddings a lo largo de la secuencia de tokens
    keras.layers.Lambda(lambda x: tf.reduce_mean(x, axis=1)),

    # Capa final para 4 clases
    keras.layers.Dense(4)
])

# Imprimir el resumen del modelo
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ text_vectorization_5                 │ (None, None)                │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_5 (Embedding)              │ (None, None, 100)           │       3,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda_5 (Lambda)                    │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 4)                   │             404 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,000,404 (11.45 MB)

 Trainable params: 3,000,404 (11.45 MB)

 Non-trainable params: 0 (0.00 B)

En la impresión del `summary`, en la columna **output shape**, la primera dimensión del tensor `None` corresponde al tamaño del minibatch, y la segunda corresponde a la longitud de la secuencia de tokens. Todas las secuencias de tokens en el minibatch tienen diferentes longitudes. Hablaremos sobre cómo manejar esto en la próxima sección.

Ahora, entrenemos la red:


In [ ]:
# Función para extraer el texto concatenado
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

# Entrenar el vectorizador
print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

# Compilar el modelo
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=['acc'])

# Entrenar el modelo
model.fit(ds_train.map(tupelize).batch(batch_size), validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 ━━━━━━━━━━━━━━━━━━━━ 38s 40ms/step - acc: 0.6763 - loss: 0.9791 - val_acc: 0.8720 - val_loss: 0.4243


> **Nota**: Estamos construyendo el vectorizador basado en un subconjunto de los datos. Esto se hace para acelerar el proceso, y podría resultar en una situación en la que no todos los tokens de nuestro texto estén presentes en el vocabulario. En este caso, esos tokens serían ignorados, lo que podría resultar en una precisión ligeramente inferior. Sin embargo, en la práctica, un subconjunto del texto suele proporcionar una buena estimación del vocabulario.


### Manejo de tamaños de secuencias variables

Vamos a entender cómo ocurre el entrenamiento en minibatches. En el ejemplo anterior, el tensor de entrada tiene una dimensión de 1, y utilizamos minibatches de longitud 128, por lo que el tamaño real del tensor es $128 \times 1$. Sin embargo, el número de tokens en cada oración es diferente. Si aplicamos la capa `TextVectorization` a una sola entrada, el número de tokens devueltos será diferente, dependiendo de cómo se tokeniza el texto:


In [ ]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


Sin embargo, cuando aplicamos el vectorizador a varias secuencias, tiene que producir un tensor de forma rectangular, por lo que llena los elementos no utilizados con el token PAD (que en nuestro caso es cero):


In [ ]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]])>

Aquí podemos ver los embeddings

In [ ]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[-0.01942449,  0.0502935 , -0.00288467, ...,  0.10349549,
          0.12438992, -0.02383408],
        [ 0.17987572, -0.19252566, -0.04424915, ...,  0.2544958 ,
          0.07159625,  0.07387033],
        [-0.01867226,  0.03623278, -0.02329332, ...,  0.00770508,
         -0.02246303, -0.03545061],
        [-0.01867226,  0.03623278, -0.02329332, ...,  0.00770508,
         -0.02246303, -0.03545061],
        [-0.01867226,  0.03623278, -0.02329332, ...,  0.00770508,
         -0.02246303, -0.03545061],
        [-0.01867226,  0.03623278, -0.02329332, ...,  0.00770508,
         -0.02246303, -0.03545061]],

       [[ 0.01328825, -0.07099735,  0.19269213, ...,  0.18524712,
          0.0646772 , -0.09691672],
        [-0.07518183,  0.10417189,  0.11709072, ...,  0.01849801,
         -0.07171822, -0.10535525],
        [-0.01942449,  0.0502935 , -0.00288467, ...,  0.10349549,
          0.12438992, -0.02383408],
        [-0.14388096,  0.07370926,  0.0090051 , ..., -0.12762024,
         -0.06

> **Nota**: Para minimizar la cantidad de padding, en algunos casos tiene sentido ordenar todas las secuencias del conjunto de datos en orden ascendente de longitud (o, más precisamente, de número de tokens). Esto garantizará que cada minibatch contenga secuencias de longitud similar.


## Embeddings semánticos: Word2Vec

En nuestro ejemplo anterior, la capa de embedding aprendió a mapear palabras a representaciones vectoriales, sin embargo, estas representaciones no tenían un significado semántico. Sería ideal aprender una representación vectorial en la que palabras similares o sinónimos correspondan a vectores que estén cerca unos de otros en términos de alguna distancia vectorial (por ejemplo, la distancia euclidiana).

Para lograr esto, necesitamos preentrenar nuestro modelo de embeddings en una gran colección de texto utilizando una técnica como [Word2Vec](https://es.wikipedia.org/wiki/Word2vec). Esta técnica se basa en dos arquitecturas principales que se utilizan para producir una representación distribuida de las palabras:

- **Bolsa continua de palabras** (CBoW), donde entrenamos el modelo para predecir una palabra a partir del contexto circundante. Dado el ngrama $(W_{-2},W_{-1},W_0,W_1,W_2)$, el objetivo del modelo es predecir $W_0$ a partir de $(W_{-2},W_{-1},W_1,W_2)$.
- **Skip-gram continuo** es opuesto a CBoW. El modelo utiliza la ventana circundante de palabras de contexto para predecir la palabra actual.

CBoW es más rápido, y aunque skip-gram es más lento, representa mejor las palabras poco frecuentes.

![Imagen que muestra los algoritmos CBoW y Skip-Gram para convertir palabras en vectores.](https://drive.google.com/uc?export=view&id=1p8hwEei0BzmcIU92uhBcvyktm8ABM31m)

Para experimentar con el embedding Word2Vec preentrenado en el conjunto de datos de Google News, podemos usar la biblioteca **gensim**. A continuación encontramos las palabras más similares a 'neural'.

> **Nota:** ¡Cuando creas los vectores de palabras por primera vez, descargarlos puede llevar algo de tiempo!


In [ ]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.780479907989502
neurons -> 0.7326499223709106
neural_circuits -> 0.7252850532531738
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923245787620544
synaptic -> 0.6699119210243225
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314660072327
neuronal_activity -> 0.6531826853752136


También podemos extraer el vector de embedding de la palabra para usarlo en el entrenamiento del modelo de clasificación. El embedding tiene 300 componentes, pero aquí solo mostramos los primeros 20 componentes del vector por claridad:


In [ ]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Lo grandioso de los embeddings semánticos es que puedes manipular la codificación vectorial basada en la semántica. Por ejemplo, podemos pedir encontrar una palabra cuya representación vectorial esté lo más cerca posible de las palabras *rey* y *mujer*, y lo más lejos posible de la palabra *hombre*:


In [ ]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118193507194519)

El ejemplo anterior utiliza algo de magia interna de GenSym, pero la lógica subyacente es bastante simple. Lo interesante de los embeddings es que puedes realizar operaciones vectoriales normales en los vectores de embeddings, y eso reflejaría operaciones sobre los **significados** de las palabras. El ejemplo anterior se puede expresar en términos de operaciones vectoriales: calculamos el vector correspondiente a **REY-HOMBRE+MUJER** (las operaciones `+` y `-` se realizan sobre las representaciones vectoriales de las palabras correspondientes), y luego encontramos la palabra más cercana en el diccionario a ese vector:


In [ ]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTA**: Tuvimos que añadir pequeños coeficientes a los vectores de *hombre* y *mujer*; intenta eliminarlos para ver qué sucede.

Para encontrar el vector más cercano, usamos las herramientas de TensorFlow para calcular un vector de distancias entre nuestro vector y todos los vectores en el vocabulario, y luego encontramos el índice de la palabra mínima utilizando `argmin`.


Aunque Word2Vec parece una excelente manera de expresar la semántica de las palabras, tiene muchas desventajas, entre ellas las siguientes:

* Tanto los modelos CBoW como skip-gram son **embeddings predictivos**, y solo consideran el contexto local. Word2Vec no aprovecha el contexto global.
* Word2Vec no tiene en cuenta la **morfología** de las palabras, es decir, el hecho de que el significado de una palabra puede depender de diferentes partes de la misma, como la raíz.

**FastText** intenta superar la segunda limitación y se basa en Word2Vec aprendiendo representaciones vectoriales para cada palabra y los n-gramas de caracteres que se encuentran dentro de cada palabra. Los valores de estas representaciones se promedian en un solo vector en cada paso de entrenamiento. Aunque esto añade mucha computación adicional al preentrenamiento, permite que los embeddings de palabras codifiquen información a nivel de subpalabra.

Otro método, **GloVe**, utiliza un enfoque diferente para los embeddings de palabras, basado en la factorización de la matriz palabra-contexto. Primero, construye una gran matriz que cuenta el número de apariciones de palabras en diferentes contextos, y luego intenta representar esta matriz en dimensiones más bajas de una manera que minimice la pérdida de reconstrucción.

La biblioteca gensim soporta estos embeddings de palabras, y puedes experimentar con ellos cambiando el código de carga del modelo mencionado anteriormente.


## Uso de embeddings preentrenados en Keras

Podemos modificar el ejemplo anterior para prepopular la matriz en nuestra capa de embedding con embeddings semánticos, como Word2Vec. Es probable que los vocabularios del embedding preentrenado y el corpus de texto no coincidan, por lo que necesitamos elegir uno. Aquí exploramos dos posibles opciones: usar el vocabulario del tokenizador y usar el vocabulario de los embeddings de Word2Vec.

### Usando el vocabulario del tokenizador

Al usar el vocabulario del tokenizador, algunas de las palabras del vocabulario tendrán embeddings correspondientes de Word2Vec, y otras faltarán. Dado que nuestro tamaño de vocabulario es `vocab_size`, y la longitud del vector de embedding de Word2Vec es `embed_size`, la capa de embedding estará representada por una matriz de pesos de forma `vocab_size`$\times$`embed_size`. Poblaremos esta matriz recorriendo el vocabulario:


In [ ]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


Para las palabras que no están presentes en el vocabulario de Word2Vec, podemos dejarlas con valores cero o generar un vector aleatorio.

Ahora podemos definir una capa de embedding con pesos preentrenados:


In [ ]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4)
])
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ text_vectorization_5                 │ (None, None)                │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_8 (Embedding)              │ ?                           │       9,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda_8 (Lambda)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,000,000 (34.33 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 9,000,000 (34.33 MB)

Ahora, entrenemos el modelo

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 ━━━━━━━━━━━━━━━━━━━━ 19s 19ms/step - acc: 0.6885 - loss: 1.2451 - val_acc: 0.8179 - val_loss: 0.9464


> **Nota**: Observa que configuramos `trainable=False` al crear la capa `Embedding`, lo que significa que no estamos reentrenando la capa de Embedding. Esto puede hacer que la precisión sea ligeramente inferior, pero acelera el entrenamiento.

### Usando el vocabulario de los embeddings

Un problema con el enfoque anterior es que los vocabularios utilizados en `TextVectorization` y `Embedding` son diferentes. Para superar este problema, podemos usar una de las siguientes soluciones:
* Reentrenar el modelo Word2Vec en nuestro vocabulario.
* Cargar nuestro conjunto de datos con el vocabulario del modelo preentrenado de Word2Vec. Los vocabularios utilizados para cargar el conjunto de datos se pueden especificar durante la carga.

El segundo enfoque parece más sencillo, así que implementémoslo. En primer lugar, crearemos una capa `TextVectorization` con el vocabulario especificado, tomado de los embeddings de Word2Vec:


In [ ]:
vocab = list(w2v.key_to_index.keys())
vectorizer = keras.layers.TextVectorization()
vectorizer.set_vocabulary(vocab)

In [ ]:
# Definir el vocabulario y la matriz de embeddings
vocab_size = len(w2v.key_to_index)  # Tamaño del vocabulario
embedding_dim = w2v.vector_size  # Dimensión de los embeddings

# Crear la matriz de embeddings a partir de los vectores de Word2Vec
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, index in w2v.key_to_index.items():
    embedding_vector = w2v[word]
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

# Definir el modelo
model = keras.models.Sequential([
    vectorizer,  # Vectorizador con el vocabulario cargado previamente
    keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim,
                           weights=[embedding_matrix], trainable=False),  # Embedding no entrenable
    keras.layers.Lambda(lambda x: tf.reduce_mean(x, axis=1)),  # Cálculo del promedio de embeddings
    keras.layers.Dense(4)  # Capa de salida para 4 clases
])

# Compilar y entrenar el modelo
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Una de las razones por las que no estamos viendo una mayor precisión es porque algunas palabras de nuestro conjunto de datos faltan en el vocabulario preentrenado de GloVe, y por lo tanto, se ignoran esencialmente. Para superar esto, podemos entrenar nuestros propios embeddings basados en nuestro conjunto de datos.


## Embeddings contextuales

Una de las principales limitaciones de las representaciones tradicionales de embeddings preentrenados, como Word2Vec, es que, aunque pueden capturar parte del significado de una palabra, no pueden diferenciar entre distintos significados. Esto puede causar problemas en los modelos posteriores.

Por ejemplo, la palabra "play" tiene diferentes significados en estas dos oraciones:
- Fui a una **obra** de teatro.
- John quiere **jugar** con sus amigos.

Los embeddings preentrenados de los que hablamos representan ambos significados de la palabra "play" en el mismo embedding. Para superar esta limitación, necesitamos construir embeddings basados en un **modelo de lenguaje**, que se entrena en un gran corpus de texto, y *sabe* cómo se pueden combinar las palabras en diferentes contextos. Discutir los embeddings contextuales está fuera del alcance de este tutorial, pero volveremos a ellos cuando hablemos de los modelos de lenguaje en la siguiente unidad.


[](https://drive.google.com/uc?export=view&id=16eURCTqgEOuhwrI59YIAQGzX3ntkUWfX)